In [1]:
from models.sos import ScalarOnScalarModel
from optimizers.nbdo import NBDO
import numpy as np

In [2]:
nbdo = NBDO(model=ScalarOnScalarModel(Kx=5, criterion = "A", order=1), latent_dim=4, seed=42,)
nbdo.compute_train_set(num_designs=1_000, runs=50)

TypeError: Can't instantiate abstract class PsiA with abstract methods loss_from_M_np, loss_from_M_tf

In [ ]:
nbdo.fit(epochs=1_000, patience=100, batch_size=512)
crit, design = nbdo.optimize(n_calls=30)

In [ ]:
crit

In [ ]:
np.round(design, 2)